In [4]:
import polars as pl
from src.paths import (
    PYTHON_VULNERABILITY_FIXES_DATA_PATH,
    PYTHON_CODE_FIXES_DATA_PATH,
    PYTHON_CODE_FIXES_WITH_CONTEXT_DATA_PATH,
    PYTHON_CODE_UNITS_DATA_PATH,
    PYTHON_CODE_CONTEXT_DATA_PATH,
)
from tqdm import tqdm
import jedi
import whatthepatch
from git import Repo
from typing import Any
import jedi.api
import jedi.common
import json

from src.process_code_changes import get_changes

## Filter out data

In [5]:
# python_commit_data = commit_data_only_top_langs.filter(pl.col("language") == "Python")
python_vulnerability_fixes = pl.read_parquet(PYTHON_VULNERABILITY_FIXES_DATA_PATH)
print(python_vulnerability_fixes.unique("vulnerability_id").shape[0])
python_vulnerability_fixes = python_vulnerability_fixes.group_by('vulnerability_id', 'repo', 'commit', 'pull_request_number', 'file', 'patch', 'patch_time', 'commit_source', 'file_extension', 'language').agg(pl.col("cwe_id"))
python_vulnerability_fixes = python_vulnerability_fixes.unique("patch")
print(python_vulnerability_fixes.unique("vulnerability_id").shape[0])


1805
1803


## Filter out data connected with test functionallity

In [6]:
python_vulnerability_fixes = python_vulnerability_fixes.filter(
    (
        pl.col("file").str.contains(r"\/{0,1}[tT][eE][sS][tT][sS]{0,1}\/")
        | pl.col("patch").str.contains("pytest")
        | pl.col("patch").str.contains("unittest")
    ).not_()
)
python_vulnerability_fixes.unique("vulnerability_id").shape[0]

1795

In [7]:
exclude_langs = [
    "txt",
    "md",
    "JSON",
    "YAML",
    "bugfix",
    "cfg",
    "rst",
    "toml",
    "lock",
    "ini",
    "in",
    "gitignore",
    "sample",
    "pem",
    "feature",
    "tif",
    "security",
    "proto",
    "conf",
    "spec",
    "bin",
    "misc",
    "pyi",
    "pxi",
    "fli",
    "gif",
    "tpl",
    "graphql",
    "http",
    "sgi",
    "pyx",
    "inc"
]
python_vulnerability_fixes = python_vulnerability_fixes.filter(
    (
        pl.col("file").str.split(".").list.last().is_in(exclude_langs)
    ).not_()
)
python_vulnerability_fixes.unique("vulnerability_id").shape[0]

1795

## Process missing commit

In [8]:
print(python_vulnerability_fixes.filter(pl.col("commit").is_null()))
python_vulnerability_fixes = python_vulnerability_fixes.with_columns(
    pl.when(pl.col("pull_request_number") == 24391)
    .then(pl.lit("86664c9405136a4904775c52e6caf100a474ec58"))
    .otherwise(pl.col("commit"))
    .alias("commit")
)
print(python_vulnerability_fixes.filter(pl.col("commit").is_null()))
# No changes related to python: https://github.com/pyca/pyopenssl/commit/6bbf44a00b35fb28df1f66aa194b2fe95eab1ab2
# Very big change: https://github.com/transifex/transifex-client/commit/e0d1f8b38ec1a24e2999d63420554d8393206f58
python_vulnerability_fixes = python_vulnerability_fixes.filter(
    ~pl.col("commit").is_in(
        [
            "6bbf44a00b35fb28df1f66aa194b2fe95eab1ab2",
            "e0d1f8b38ec1a24e2999d63420554d8393206f58",
        ]
    )
)
excluded_vulns = [
    "2024-34702",
    "2022-29198",
    "2022-41909",
    "2022-41891",
    "2016-0740",
    "2024-21485",
    "2024-8948",
    "2018-10861",
    "2022-4526",
    "2022-31116",
    "2023-52266",
    "2021-28359",
    "2022-25882",
    "2024-32979",
]
python_vulnerability_fixes = python_vulnerability_fixes.filter(
    pl.col("vulnerability_id").is_in(excluded_vulns).not_(),
    pl.col("file") != "setup.py"
)
print(python_vulnerability_fixes.filter(pl.col("commit").is_null()))

shape: (1, 11)
┌────────────┬────────────┬────────┬────────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ vulnerabil ┆ repo       ┆ commit ┆ pull_reque ┆ … ┆ commit_so ┆ file_exte ┆ language ┆ cwe_id    │
│ ity_id     ┆ ---        ┆ ---    ┆ st_number  ┆   ┆ urce      ┆ nsion     ┆ ---      ┆ ---       │
│ ---        ┆ str        ┆ str    ┆ ---        ┆   ┆ ---       ┆ ---       ┆ str      ┆ list[str] │
│ str        ┆            ┆        ┆ i64        ┆   ┆ str       ┆ str       ┆          ┆           │
╞════════════╪════════════╪════════╪════════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ 2021-3733  ┆ python/cpy ┆ null   ┆ 24391      ┆ … ┆ github    ┆ py        ┆ Python   ┆ ["CWE-400 │
│            ┆ thon       ┆        ┆            ┆   ┆           ┆           ┆          ┆ "]        │
└────────────┴────────────┴────────┴────────────┴───┴───────────┴───────────┴──────────┴───────────┘
shape: (0, 11)
┌─────────────┬──────┬────────┬─────────────┬───┬────────────

In [9]:
python_vulnerability_fixes.filter(pl.col("vulnerability_id") == "2022-29198").unique("language")

vulnerability_id,repo,commit,pull_request_number,file,patch,patch_time,commit_source,file_extension,language,cwe_id
str,str,str,i64,str,str,"datetime[μs, UTC]",str,str,str,list[str]


In [10]:
import logging

if PYTHON_CODE_FIXES_DATA_PATH.exists():
    print("Reading code fixes")
    code_unit_changes = pl.read_parquet(PYTHON_CODE_FIXES_DATA_PATH).to_dicts()
else:
    code_unit_changes: list[dict[str, Any]] = []

repos: dict[str, Repo] = {}

grouped_vulnerabilities = (
    python_vulnerability_fixes.group_by(
        "repo", "vulnerability_id", "commit", "commit_source", "cwe_id"
    )
    .agg(pl.col("patch"), pl.col("file"), pl.col("language"))
    .sample(fraction=1, shuffle=True)
)
errors: list[dict[str, Any]] = []
checked_commits = set([change["commit"] for change in code_unit_changes])


vulnerabilities_to_check = grouped_vulnerabilities.to_dicts()


for commit_data_row in tqdm(vulnerabilities_to_check):
    if commit_data_row["commit"] in checked_commits:
        continue
    try:
        get_changes(commit_data_row)
    except Exception as e:
        logging.exception(f"Error in {commit_data_row}")

Reading code fixes


  0%|          | 8/2302 [00:02<12:36,  3.03it/s]


KeyboardInterrupt: 

In [11]:
new_code_unit_changes = []
new_code_context_changes = []

for commit_data_file in PYTHON_CODE_UNITS_DATA_PATH.rglob("*.json"):
    try:
        with commit_data_file.open() as f:
            new_code_unit_changes.append(json.load(f))
    except:
        print(commit_data_file)
        commit_data_file.unlink()
for commit_data_file in PYTHON_CODE_CONTEXT_DATA_PATH.rglob("*.json"):
    try:
        with commit_data_file.open() as f:
            new_code_context_changes.append(json.load(f))
    except:
        print(commit_data_file)
        commit_data_file.unlink()

pl.DataFrame(new_code_unit_changes).write_parquet(PYTHON_CODE_FIXES_DATA_PATH)
pl.DataFrame(new_code_context_changes).write_parquet(PYTHON_CODE_FIXES_WITH_CONTEXT_DATA_PATH)

In [12]:
code_unit_changes_df = pl.read_parquet(PYTHON_CODE_FIXES_DATA_PATH)
code_unit_changes_df = code_unit_changes_df.filter(
    pl.col("vulnerability_id").is_in(excluded_vulns).not_(),
    pl.col("new_file") != "setup.py",
    pl.col("old_file") != "setup.py"
).with_columns(
    pl.col("code_unit_after_fix").str.replace_all(r"\n\s*\n", "\n").alias("code_unit_after_fix"),
    pl.col("code_unit_before_fix").str.replace_all(r"\n\s*\n", "\n").alias("code_unit_before_fix"),
)
code_unit_changes_df.describe()

statistic,commit,repo,new_file,patch,code_unit_after_fix,vulnerability_id,cwe_id,old_file,code_unit_before_fix
str,str,str,str,str,str,str,f64,str,str
"""count""","""4531""","""4531""","""4531""","""4531""","""4531""","""4531""",4531.0,"""4531""","""4531"""
"""null_count""","""0""","""0""","""0""","""0""","""0""","""0""",0.0,"""0""","""0"""
"""mean""",null,null,null,null,null,null,null,null,null
"""std""",null,null,null,null,null,null,null,null,null
"""min""","""001b0634cd309e372edb6d7d95d083…","""389ds/389-ds-base""",""".devcontainer/library-scripts/…","""@@ -1 +1 @@ -__version__ = ""0.…","""""","""2013-0208""",null,""".devcontainer/library-scripts/…",""""""
"""25%""",null,null,null,null,null,null,null,null,null
"""50%""",null,null,null,null,null,null,null,null,null
"""75%""",null,null,null,null,null,null,null,null,null
"""max""","""ffd1d62dd50d5f4190013b39cedcdf…","""zwczou/weixin-python""","""zproject/urls.py""","""@@ -99,7 +99,9 @@ extern ""C"" {…","""{{ community.description | san…","""GHSA-x563-6hqv-26mr""",null,"""zproject/urls.py""","""{{ community.description|safe …"


In [ ]:

code_unit_changes_df.with_columns(
    pl.col("new_file").str.split(".").list.last().alias("lang")
).group_by("vulnerability_id").agg("lang").with_columns(pl.col("lang").list.unique()).unique("lang").filter(pl.col("lang").list.contains("py").not_()).select("vulnerability_id").to_series().to_list()

['2022-41909',
 '2022-41891',
 '2016-0740',
 '2024-21485',
 '2024-8948',
 '2018-10861',
 '2022-4526',
 '2022-31116',
 '2023-52266',
 '2021-28359',
 '2022-25882',
 '2024-32979']

In [ ]:
total = 0
errors_count = 0
bad_commits = set()
for row in code_unit_changes_df.sample(fraction=1, shuffle=True).iter_rows(named=True):
    file_extension = row["new_file"].split(".")[-1]
    if file_extension in {"py", "pyi", "pyx", "pxi"}:
        total += 1
        script = jedi.Script(code=row["code_unit_after_fix"])
        errors = script.get_syntax_errors()
        if errors:
            bad_commits.add(row["commit"])
            errors_count += 1


KeyboardInterrupt: 

In [13]:
for commit_data_row in vulnerabilities_to_check:
    if commit_data_row["commit"] == "32de2154ef9f946160e5dc01a4d8a449dd0bd259":
        get_changes(commit_data_row)